In [1]:
import torchio as tio
from torchvision.transforms import Compose

def examination_to_tensor_vista3d(data_path):
    RESAMPLE_TO_SPACING = (1.5, 1.5, 1.5)
    A_MIN = -963.8247715525971
    A_MAX = 1053.678477684517
    B_MIN = 0.0
    B_MAX = 1.0

    transforms = Compose([
        Compose([
            tio.Clamp(out_min=A_MIN, out_max=A_MAX),
            tio.RescaleIntensity(
                out_min_max=(B_MIN, B_MAX),
                in_min_max=(A_MIN, A_MAX),
            )
        ]),
        tio.ToCanonical(), # RAS+
        tio.Resample(target=RESAMPLE_TO_SPACING, antialias=True),
        tio.Lambda(lambda x: x.float())
    ])
    image = tio.ScalarImage(data_path)
    image = transforms(image)
    return image.data

/home/borntowarn/projects/chest-diseases/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from monai.transforms import (
    Compose,
    LoadImaged,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    CastToTyped,
    EnsureChannelFirstd
)

resample_to_spacing = (1.5, 1.5, 1.5)


monai_transforms = Compose([
    LoadImaged(
        keys=["image"],
        image_only=True,
    ),

    EnsureChannelFirstd(
        keys=["image"],
    ),

    ScaleIntensityRanged(
        keys="image",
        a_min=-963.8247715525971,
        a_max=1053.678477684517,
        b_min=0.0,
        b_max=1.0,
        clip=True
    ),

    Orientationd(
        keys=["image"],
        axcodes="RAS"
    ),

    Spacingd(
        keys=["image"],
        pixdim=resample_to_spacing,
        mode=["bilinear"],
        align_corners=True
    ),

    CastToTyped(
        keys=["image"],
        dtype=[torch.float32]
    )
])

/home/borntowarn/projects/chest-diseases/venv/lib/python3.11/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
  warn_deprecated(argname, msg, warning_category)


In [ ]:
parser = ConfigParser()
parser.read_config('/home/borntowarn/projects/chest-diseases/training/vista3d/configs/infer.yaml')

In [ ]:
infer_transforms = parser.get_parsed_content("transforms_infer")

In [ ]:
infer_transforms = parser.get_parsed_content("transforms_infer")

file = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed/valid_1/valid_1_a/valid_1_a_1.nii.gz'
tensor2 = infer_transforms({"image": file})
tensor1 = examination_to_tensor_vista3d(file)

In [3]:
file = '/home/borntowarn/projects/chest-diseases/training/data/CT-RATE/dataset/valid_fixed/valid_1/valid_1_a/valid_1_a_1.nii.gz'
tensor3 = monai_transforms({"image": file})


In [10]:
tensor3['image'].as_tensor()

tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000

In [ ]:
tensor1.shape, tensor2["image"].shape, tensor3["image"].shape

(torch.Size([1, 233, 233, 251]),
 torch.Size([1, 234, 234, 251]),
 torch.Size([1, 234, 234, 251]))

In [ ]:
torch.allclose(tensor3["image"], tensor2["image"])

True

In [ ]:
tensor3["image"]

metatensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.

In [ ]:
tensor2["image"]

metatensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]],

         [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.